Derived from sqlite_P1_v10

In [4]:
import pandas as pd
import sqlite3

In [5]:
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 10)

In [6]:
p_pfad=r'O:\Work\DV\MatthiasH\BBG_Tracking'.replace(chr(92),'/')
p_ddf=p_pfad+'/'+'R315114383A_completes.ddf'


In [7]:
con = sqlite3.connect(p_ddf)
levels = pd.read_sql_query("SELECT * FROM Levels ", con)
con.close()

In [8]:
levels

TableNumber TableName ParentName DSCTableName
0             1        L1       None        HDATA
1             2        L2         L1           S8
2             3        L3         L1    DummyQ1_1
3             4        L4         L1    DummyQ1_2
4             5        L5         L1           Q1
..          ...       ...        ...          ...
20           21       L21        L20         Q17a
21           22       L22        L20          Q18
22           23       L23         L1         MDN1
23           24       L24         L1         MDN2
24           25       L25         L1           E1

[25 rows x 4 columns]

In [15]:
Parent_L1_Tables=levels[(levels.ParentName=='L1')&(levels.DSCTableName.str[-5:]!='_bans')&(levels.DSCTableName.str[:5]!='Dummy')].DSCTableName.copy()
Parent_L1_Tables=Parent_L1_Tables.reset_index(drop=True)
Parent_L20_Tables=levels[(levels.ParentName=='L20')&(levels.DSCTableName.str[-5:]!='_bans')&(levels.DSCTableName.str[:5]!='Dummy')].DSCTableName.copy()
Parent_L20_Tables=Parent_L20_Tables.reset_index(drop=True)

In [16]:
Parent_L20_Tables

0    Q17a
1     Q18
Name: DSCTableName, dtype: object

In [18]:
dfL1=pd.DataFrame([])

for TableName in Parent_L1_Tables.values.tolist():
    #create dictionary from mdd-info to replace column names with their full qualified name
    #dict_VarL=dict(zip(mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable.str[len(TableName)+5:],
             #mys[mys.Variable.str[:len(TableName)+5]==TableName+'[..].'].Variable))
    dict_levels=dict(zip(levels.DSCTableName,levels.TableName))
    TableName.replace(TableName,dict_levels[TableName])
    # Read sqlite query results into a pandas DataFrame
    con = sqlite3.connect(p_ddf)
    Table=TableName.replace(TableName,dict_levels[TableName])
    df = pd.read_sql_query("SELECT [Respondent.Serial:L] as Respondent, "+Table+".* from "+Table+"  JOIN L1 ON L1.[:P0]="+Table+".[:P1] ", con)
    con.close()
    #Seperate :S (Multi) and :C1 (Single) Variables 
    dfs=df[df.columns[df.columns.str.contains(':S$|Respondent|LevelId')]].copy()
    dfc1=df[df.columns[df.columns.str.contains(':C1$|Respondent')]].copy()

    #Exclude banner variables
    if 'ban:S' in dfs.columns:
        dfs=dfs.drop('ban:S', axis=1)
    #Create dictionary to strip :C/:C1 Extensions
    dict_c1=dict(zip(dfc1.columns,dfc1.columns.str.replace('(:C1|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfc1.rename(columns=dict_c1, inplace=True)
    #dfc1.rename(columns=dict_VarL, inplace=True)
    #Add column with TableName as constant value
    dfc1['Level']=TableName
    #Bring single punch data in long format:
    dfc1m=pd.melt(dfc1,id_vars=['Respondent','Level','LevelId'],var_name='Variable', value_name='Value')
    #Add column with TableName as constant value
    #dfc1m['Level']=TableName
    #Add column to indicate info type as 'Single'
    dfc1m['Type']='Single'

    dict_s=dict(zip(dfs.columns,dfs.columns.str.replace('(:C1|:S|\.Serial:L)','')))
    #Apply dictionaries to standardise column names according to mdd-info
    dfs.rename(columns=dict_s, inplace=True)
    #dfs.rename(columns=dict_VarL, inplace=True)
    dfs.loc[:,'Level']=TableName
    #Bring multi punch data in long format:
    dfsm=pd.melt(dfs,id_vars=['Respondent','Level','LevelId'],var_name='Variable', value_name='Value')
    dfsmt=pd.DataFrame([])
    if dfsm.Respondent.count()>0:
        dfsmt=dfsm.drop(['Value'], axis=1).join(dfsm['Value'].str.split(';',expand=True).stack().reset_index(drop=True, level=1).rename('Value') )
        dfsmt=dfsmt[dfsmt.Value>' ']
        dfsmt['Value']=dfsmt['Value'].astype(int)
        dfsmt['Respondent']=dfsmt['Respondent'].astype(int)
        dfsmt['LevelId']=dfsmt['LevelId'].astype(int)
        dfsmt['Type']='Multi'

    dfL1=pd.concat([dfL1,dfc1m])
    dfL1=pd.concat([dfL1,dfsmt])


In [19]:
dfL1

Respondent Level  LevelId Variable  Value    Type
0             12    S8      184    slice  183.0  Single
1             12    S8      185    slice  183.0  Single
2             15    S8      184    slice  185.0  Single
3             20    S8      184    slice  182.0  Single
4             22    S8      184    slice  183.0  Single
...          ...   ...      ...      ...    ...     ...
7519        7947    E1      186    slice  183.0  Single
7520        7947    E1      187    slice  185.0  Single
7521        7947    E1      188    slice  185.0  Single
7522        7947    E1      189    slice  184.0  Single
7523        7947    E1      195    slice  184.0  Single

[147385 rows x 6 columns]

In [12]:
dfL1

Respondent         Level  LevelId                 Variable  Value  \
0             206    LoB_Brands      142  LoB_Brands[..].Provider    142   
1             206    LoB_Brands      119  LoB_Brands[..].Provider    250   
2             206    LoB_Brands      154  LoB_Brands[..].Provider    250   
3             206    LoB_Brands      155  LoB_Brands[..].Provider    250   
4             206    LoB_Brands      241  LoB_Brands[..].Provider    250   
...           ...           ...      ...                      ...    ...   
14084        7577  Image_Brands      154     Image_Brands[..].Q16    244   
14084        7577  Image_Brands      154     Image_Brands[..].Q16    252   
14084        7577  Image_Brands      154     Image_Brands[..].Q16    119   
14084        7577  Image_Brands      154     Image_Brands[..].Q16    241   
14084        7577  Image_Brands      154     Image_Brands[..].Q16    242   

         Type  
0      Single  
1      Single  
2      Single  
3      Single  
4      Single  
...       ...  
14084   Multi  
14084   Multi  
14084   Multi  
14084   Multi  
14084   Multi  

[433218 rows x 6 columns]

In [17]:
df1=pd.merge(left=dfL1, right=mysCategory, how='left', left_on=['Variable','Value'], right_on=['Variable','Value'])
df=pd.merge(left=df1, right=mysBTS, how='left', left_on=['Level','LevelId'], right_on=['Level','LevelId'])


In [18]:
df[['Respondent','BrandCode', 'BrandLabel','Variable','CategoryLabel', 'ID_BTS_1','ID_BTS_2','ID_CrossLOB' ]]

Respondent BrandCode     BrandLabel                 Variable  \
0              206        _1   Lob Brand 01  LoB_Brands[..].Provider   
1              206       _10   Lob Brand 10  LoB_Brands[..].Provider   
2              206        _2   Lob Brand 02  LoB_Brands[..].Provider   
3              206        _3   Lob Brand 03  LoB_Brands[..].Provider   
4              206        _4   Lob Brand 04  LoB_Brands[..].Provider   
...            ...       ...            ...                      ...   
433213        7577        _2  AIG Insurance     Image_Brands[..].Q16   
433214        7577        _2  AIG Insurance     Image_Brands[..].Q16   
433215        7577        _2  AIG Insurance     Image_Brands[..].Q16   
433216        7577        _2  AIG Insurance     Image_Brands[..].Q16   
433217        7577        _2  AIG Insurance     Image_Brands[..].Q16   

                                   CategoryLabel  ID_BTS_1  ID_BTS_2  \
0                                            Yes       NaN       NaN   
1                                             No       NaN       NaN   
2                                             No       NaN       NaN   
3                                             No       NaN       NaN   
4                                             No       NaN       NaN   
...                                          ...       ...       ...   
433213                      Provides good advice   50115.0   50214.0   
433214  Is the right partner in decisive moments   50115.0   50214.0   
433215           Is a financially stable company   50115.0   50214.0   
433216                        Is worth the money   50115.0   50214.0   
433217       Provides a high standard of service   50115.0   50214.0   

        ID_CrossLOB  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
...             ...  
433213          NaN  
433214          NaN  
433215          NaN  
433216          NaN  
433217          NaN  

[433218 rows x 8 columns]

In [19]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("data/clean3.DDF")
dfL0 = pd.read_sql_query("SELECT * FROM L1", con)
con.close()

In [20]:
dict_Value_Wave=dict(zip(mys[mys.Variable.str.contains('[Ww]ave')].Value,mys[mys.Variable.str.contains('[Ww]ave')].CategoryName.str[1:].astype(int)))
dict_Value_Country=dict(zip(mys[mys.Variable.str.contains('^Country$')].Value,mys[mys.Variable.str.contains('Country')].CategoryName))
dict_Value_LOB=dict(zip(mys[mys.Variable.str.contains('^L[oO]B$')].Value.astype(str).str[:3]+';',mys[mys.Variable.str.contains('^L[oO]B$')].CategoryName))

In [21]:
dict_Value_LOB

{'142;': '_1', '154;': '_2'}

In [22]:
dfL0['Country:C1'].replace(dict_Value_Country)
dfL0['Wave:C1'].replace(dict_Value_Wave)
dfL0['LOB:S'].replace(dict_Value_LOB)

0       _1
1       _2
2       _1
3       _1
4       _1
        ..
1276    _2
1277    _2
1278    _2
1279    _2
1280    _2
Name: LOB:S, dtype: object

In [23]:
mys[mys.Variable.str.contains('Country')]

Variable CategoryName  Value        CategoryLabel  ID_CrossLOB  ID_BTS_1  \
3   Country          POL    101               Poland          NaN       NaN   
4   Country          CZE     93       Czech Republic          NaN       NaN   
5   Country           _3    155             Slovakia          NaN       NaN   
6   Country          ROM   1520              Romania          NaN       NaN   
7   Country          MYS     96             Malaysia          NaN       NaN   
..      ...          ...    ...                  ...          ...       ...   
38  Country          _91    710     Australia Sydney          NaN       NaN   
39  Country          _92    711  Australia Melbourne          NaN       NaN   
40  Country          _93    712   Australia Brisbane          NaN       NaN   
41  Country          _94    713      Australia Perth          NaN       NaN   
42  Country          _95    714   Australia Adelaide          NaN       NaN   

    ID_BTS_2  
3        NaN  
4        NaN  
5        NaN  
6        NaN  
7        NaN  
..       ...  
38       NaN  
39       NaN  
40       NaN  
41       NaN  
42       NaN  

[40 rows x 7 columns]

In [24]:
dfL0

:P0  Respondent.Serial:L DataCollection.Status:S  \
0        1                  206                   6;13;   
1        2                  269                   6;13;   
2        3                  420                   6;13;   
3        4                  460                   6;13;   
4        5                  498                   6;13;   
...    ...                  ...                     ...   
1276  1277                 7266                   6;13;   
1277  1278                 7270                   6;13;   
1278  1279                 7372                   6;13;   
1279  1280                 7375                   6;13;   
1280  1281                 7577                   6;13;   

      DataCollection.StartTime:T  DataCollection.FinishTime:T  sprachech1:C1  \
0                   42576.578021                 42576.581782           1210   
1                   42576.922789                 42576.936319           1211   
2                   42577.251678                 42577.262234           1209   
3                   42577.398171                 42577.405405           1211   
4                   42577.518507                 42577.586979           1211   
...                          ...                          ...            ...   
1276                42598.521470                 42598.539514           1210   
1277                42598.528333                 42598.536956           1211   
1278                42598.757708                 42598.792928           1211   
1279                42598.878692                 42598.890683           1209   
1280                42599.279896                 42599.305359           1209   

                  ...               Coded.Q6:S Coded.Q7a_Top:C1  \
0                 ...                     142;              142   
1                 ...                     196;             1607   
2                 ...                     251;              251   
3                 ...                     242;              154   
4                 ...                     142;              242   
...               ...                      ...              ...   
1276              ...                     155;              244   
1277              ...                    1603;             1620   
1278              ...               1602;1603;             1601   
1279              ...                     244;              196   
1280              ...                    1628;             1616   

                                Coded.Q7a:S  Coded.Q11:C1  \
0                                      142;           142   
1                           1607;1601;1603;          1603   
2                                      251;           251   
3                                  154;242;           154   
4                                  242;142;           251   
...                                     ...           ...   
1276                          244;1603;155;           155   
1277                        1620;1603;1628;           244   
1278  1601;244;1603;1607;1628;119;252;1602;           119   
1279                                   244;           244   
1280                        1616;1603;1628;          1603   

                   Coded.Q12:S                Coded.Q11Q12:S  
0                         196;                          142;  
1                         196;                         1603;  
2                         142;                      251;142;  
3                         196;                          154;  
4                     142;242;                  251;142;242;  
...                        ...                           ...  
1276                 244;1603;                 155;244;1603;  
1277            1603;1628;155;            244;1603;1628;155;  
1278  1602;1603;1607;1628;241;  119;1602;1603;1607;1628;241;  
1279                      196;                          244;  
1280                      196;                         1603;  

[1281 rows x 91 columns]

p_pfad=r'O:\Work\DV\MatthiasH\jupyter\data'.replace(chr(92),'/')
p_file='L1.csv'
p_tab='L1'
df.to_csv(p_pfad+'/'+p_file, sep='\t', index=False)